In [1]:
from __future__ import print_function
import keras
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D,AveragePooling2D
from keras import backend as K
num_classes = 10
epochs = 12

Using TensorFlow backend.


In [92]:
def makeFixed(Arr):
    Arr2 = np.zeros_like(Arr)
    for i in range(1,6):
        for j in range(0,len(Arr)):
            if(abs(Arr[j]) >= np.power(2,-i,dtype = float)):
                if(Arr[j] > 0):
                    Arr2[j] += np.power(2,-i,dtype = float)
                    Arr[j] -= np.power(2,-i,dtype = float)
                if(Arr[j] < 0):
                    Arr2[j] -= np.power(2,-i,dtype = float)
                    Arr[j] += np.power(2,-i,dtype = float)
    return Arr2

[ 0.5    -0.3125]


In [97]:
# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(1, kernel_size=(3, 3), padding = 'valid',strides = 1, use_bias = True,
                 input_shape=input_shape,activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2),strides = 2))
model.add(Flatten())
model.add(Dense(num_classes, use_bias = True))
model.add(Activation('softmax'))


model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          epochs=1,
          verbose=1,
          batch_size=20,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 18s 293us/step - loss: 0.4338 - acc: 0.8722 - val_loss: 0.3129 - val_acc: 0.9112
Test loss: 0.31291502078771594
Test accuracy: 0.9112


In [98]:
import numpy as np
model2 = np.copy(model)
for layer in model.layers:
    weights = np.array(layer.get_weights()) # list of numpy arrays
    if(len(weights) >= 2):
        print(weights[1].flatten())
        shape1 = weights[1].shape
        weights1 = weights[1].flatten()
        weights3 = makeFixed(weights1)
        weights3 = np.resize(weights3,shape1)
        shape2 = weights[0].shape
        weights2 = weights[0].flatten()
        weights4 = makeFixed(weights2)
        weights4 = np.resize(weights4,shape2)
        weights[0] = weights4
        weights[1] = weights3
        print(weights[1].flatten())
        layer.set_weights(weights)
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
model = model2

[0.04110432]
[0.03125]
[-0.12147353  0.1029686  -0.0028596  -0.08939906 -0.00389476  0.27986223
 -0.05701626  0.11593188 -0.31324014 -0.11279692]
[-0.09375  0.09375  0.      -0.0625   0.       0.25    -0.03125  0.09375
 -0.3125  -0.09375]
Test loss: 0.33703186528086665
Test accuracy: 0.9041
